In [ ]:
# Install required packages
# Run this cell first if you don't have these packages installed
!pip install numpy matplotlib scikit-learn datasets torch


# Week 7 — Convolutional Neural Networks (Image Classification) - PyTorch Version

**Setup Instructions:**
1. Run the cell above to install required packages (if not already installed)
2. **Important:** If you get import errors after installing packages, restart the Jupyter kernel:
   - Go to `Kernel` → `Restart Kernel` in the menu
   - Or use the restart button in the toolbar
3. Then re-run the import cell below

**Note:** This notebook uses PyTorch instead of NumPy for CNN implementation. PyTorch provides automatic differentiation and GPU support, making it easier to build and train neural networks.

**Objectives**

- Load and visualize the CIFAR-100 dataset from HuggingFace.
- Build a PCA + Logistic Regression baseline classifier; compute evaluation metrics (e.g., accuracy).
- Implement a simple CNN classifier using PyTorch that barely beats or fails to beat the baseline.
- Build a deeper CNN model that achieves better performance.
- Train a CNN model on data‑augmented images and visualize augmentations.
- Explore an advanced CNN feature (e.g., global average pooling) and observe its impact.


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from utils import (
    show_result,
    load_cifar100_dataset,
    pca_logistic_baseline,
    test_exercise_7_pca,
    test_exercise_7_simple_cnn,
    test_exercise_7_proper_cnn,
    test_exercise_7_data_aug_cnn,
    test_exercise_7_advanced_cnn,
    accuracy
)

# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 1. CIFAR-100 Image Dataset

In this exercise, we'll use the CIFAR-100 dataset, a well-known benchmark dataset for image classification. CIFAR-100 contains 60,000 32×32 color images in 100 classes (100 fine-grained classes), with 6,000 images per class.

We'll load the dataset from HuggingFace and convert it to grayscale to simplify training and focus on the CNN architecture rather than computational complexity.

**Task:** Use the `load_cifar10_dataset` function from `utils.py` to load a subset of CIFAR-100, then visualize a few random samples from each class. Report the number of training and test examples.

**Hint:** Call `load_cifar10_dataset(n_train=50000, n_test=200, seed=0, grayscale=False)` to get a manageable subset. The function returns 5 values: train images, train labels, test images, test labels, and class names.


In [ ]:
# Load CIFAR-100 dataset
# Using the full dataset for high performance
X_train, y_train, X_test, y_test, class_names = load_cifar100_dataset(
    n_train=50000, n_test=10000, seed=0, grayscale=False
)
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
print(f"Image shape: {X_train.shape[1:]}")
print(f"Classes: {class_names}")

# Visualize a few random samples from the training set
fig, axes = plt.subplots(1, 5, figsize=(12, 2.5))
for ax in axes:
    idx = random.randint(0, len(X_train) - 1)
    ax.imshow(X_train[idx]) # RGB, no cmap needed
    ax.set_title(f"{class_names[y_train[idx]]}\n(label {y_train[idx]})")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. PCA + Logistic Regression Baseline

A simple yet strong baseline for image classification is to flatten each image into a vector, project it onto a lower‑dimensional subspace using **Principal Component Analysis (PCA)**, and then train a multinomial logistic regression classifier.

1. Flatten the training and test images (shape `(N, H*W)`).
2. Fit a PCA model on the training data and project both the training and test data into a lower‑dimensional space (e.g., 20 components).
3. Train a `LogisticRegression` classifier on the reduced features.
4. Evaluate the classifier using **accuracy** (the fraction of correct predictions).

**Task:** Complete the function `student_pca_baseline(...)` below to implement this baseline. It should return the test accuracy as a float in `[0,1]`.

**Hints:**
- Use `train_images.reshape(train_images.shape[0], -1)` to flatten images from (N, H, W) to (N, H*W)
- Import `PCA` from `sklearn.decomposition` and `LogisticRegression` from `sklearn.linear_model`
- Make sure `n_components` doesn't exceed the number of features (use `min(n_components, n_features)`)
- Use `pca.fit_transform()` for training data and `pca.transform()` for test data
- The `accuracy` function from `utils` computes the classification accuracy


In [ ]:
def student_pca_baseline(train_images, train_labels, test_images, test_labels, n_components=20):
    '''
    Implements a PCA + Logistic Regression baseline classifier.

    Parameters:
        train_images: numpy array of shape (N_train, H, W) with float32 values in [0,1].
        train_labels: numpy array of shape (N_train,) of integer labels.
        test_images: numpy array of shape (N_test, H, W).
        test_labels: numpy array of shape (N_test,).
        n_components: number of principal components to retain.

    Returns:
        Test accuracy as a float in [0,1].
    '''
    # TODO: flatten images, fit PCA, train LogisticRegression, compute accuracy
    raise NotImplementedError


In [ ]:
# Evaluate the PCA baseline implementation
res = test_exercise_7_pca(student_pca_baseline)
show_result("Exercise 1 – PCA Baseline", res)

# If implemented, you can also test on the dataset generated above
try:
    acc = student_pca_baseline(X_train, y_train, X_test, y_test, 20)
    print(f"PCA baseline accuracy on the synthetic dataset: {acc:.3f}")
except NotImplementedError:
    print("Implement student_pca_baseline above.")


## 3. Simple Convolutional Neural Network (PyTorch)

Convolutional neural networks (CNNs) process images by learning **filters** that extract local patterns. We'll start with a very small CNN using PyTorch:

- One convolutional layer with a few filters (e.g., 4 filters, each $3\times3$).
- Apply a non‑linear activation such as ReLU.
- Flatten the result and feed it into a linear layer to produce class logits.

For training, use cross‑entropy loss and an optimizer (e.g., SGD) for a few epochs. Because this network is very shallow and the dataset is small, it may perform worse than the PCA baseline.

**Task:** Complete the function `student_simple_cnn(...)` below. It should construct the described network using PyTorch's `nn.Module`, train it for a few epochs on the training set, and return the test accuracy.

**Hints:**
- Use `nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3)` for the convolutional layer
- Use `nn.ReLU()` for activation
- Use `nn.Flatten()` or `.view()` to flatten the feature maps
- Use `nn.Linear()` for the final classification layer
- Convert NumPy arrays to PyTorch tensors: `torch.from_numpy(arr).float()`
- Move tensors to device: `.to(device)`
- Use `nn.CrossEntropyLoss()` for loss and `optim.SGD()` for optimizer
- Remember to call `model.train()` during training and `model.eval()` during evaluation


In [ ]:
def student_simple_cnn(train_images, train_labels, test_images, test_labels, num_epochs=5, learning_rate=0.01):
    '''
    Build and train a simple CNN with one convolutional layer followed by a linear classifier using PyTorch.
    Use small filter sizes (e.g., 3x3) and a small number of filters (e.g., 4).

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy as a float.
    '''
    # TODO: implement PyTorch CNN model, training loop, and evaluation
    raise NotImplementedError


In [ ]:
# Evaluate the simple CNN implementation
res = test_exercise_7_simple_cnn(student_simple_cnn)
show_result("Exercise 2 – Simple CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_simple_cnn(X_train, y_train, X_test, y_test)
    print(f"Simple CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_simple_cnn above.")


## 4. Deeper CNN (Improved Model)

Now extend your network to be **deeper** with multiple convolutional blocks. Your goal is to achieve **>50% accuracy on CIFAR-100**.

**Suggested Architecture** (you can modify this):
- **Block 1:** [Conv(3→64) → BN → ReLU → Conv(64→64) → BN → ReLU] → MaxPool(2×2) → Dropout(0.1)
- **Block 2:** [Conv(64→128) → BN → ReLU → Conv(128→128) → BN → ReLU] → MaxPool(2×2) → Dropout(0.2)
- **Block 3:** [Conv(128→256) → BN → ReLU → Conv(256→256) → BN → ReLU] → MaxPool(2×2) → Dropout(0.3)
- **Classifier:** Flatten → Linear(256×4×4 → 512) → BN → ReLU → Dropout(0.5) → Linear(512 → 100)

**Key Components:**
1. **Batch Normalization** (`nn.BatchNorm2d`): Normalizes activations, speeds up training
2. **Dropout** (`nn.Dropout2d` for conv layers, `nn.Dropout` for fc layers): Prevents overfitting
3. **Data Augmentation**: Apply random transformations during training
4. **Mini-batch Training**: Use `DataLoader` to train in batches

**Task:** Complete the function `student_proper_cnn(...)` below.

**Hints:**
- Use `padding=1` in `Conv2d` to preserve spatial dimensions (32×32 → 32×32)
- Use `nn.MaxPool2d(2, 2)` to downsample by 2x (32×32 → 16×16 → 8×8 → 4×4)
- After 3 pooling layers: 32÷2÷2÷2 = 4, so final feature map is 4×4
- Progressive dropout: 0.1 → 0.2 → 0.3 → 0.5 (less in early layers, more in later layers)
- **Data Augmentation** (apply during training):
  - Random horizontal flip: `torch.flip(img, dims=[3])`
  - Random crop: `F.pad(img, (4,4,4,4), mode='reflect')` then crop 32×32
- **Weight Initialization**: Use Kaiming (He) initialization for ReLU networks
  ```python
  nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
  ```
- **Optimizer**: `optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)`
- **Scheduler**: `CosineAnnealingLR` for smooth learning rate decay
- **Loss**: `nn.CrossEntropyLoss(label_smoothing=0.1)` prevents overconfidence
- **Gradient Clipping**: `torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)`
- **Training**: 50 epochs with batch_size=64 should achieve >50% accuracy
- **Mini-batches**: Use `TensorDataset` and `DataLoader` for efficient batch training

**Example Structure:**
```python
class ProperCNN(nn.Module):
    def __init__(self):
        super(ProperCNN, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.1)
        # ... continue with blocks 2, 3, and classifier
```


In [ ]:
def student_proper_cnn(train_images, train_labels, test_images, test_labels, num_epochs=50, learning_rate=0.001, batch_size=64):
    '''
    Build and train a moderately deep CNN optimized for CIFAR-100.
    Goal: Achieve >50% test accuracy.

    Steps:
    1. Define a CNN with 3 conv blocks (each with 2 convs, BN, ReLU, MaxPool, Dropout)
    2. Add a classifier (Flatten → FC → BN → ReLU → Dropout → FC)
    3. Create data augmentation function (horizontal flip + random crop)
    4. Set up DataLoader for mini-batch training
    5. Initialize weights with Kaiming initialization
    6. Use AdamW optimizer + CosineAnnealingLR scheduler
    7. Train with gradient clipping
    8. Evaluate on test set

    Parameters:
        train_images: numpy array (N_train, H, W, C) with values in [0,1].
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W, C).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs (default: 50).
        learning_rate: initial learning rate (default: 0.001).
        batch_size: mini-batch size (default: 64).

    Returns:
        Test accuracy as a float in [0,1].
    '''
    n_classes = len(np.unique(train_labels))
    from torch.utils.data import TensorDataset, DataLoader
    
    # TODO: Define your ProperCNN model class
    # class ProperCNN(nn.Module):
    #     def __init__(self):
    #         super(ProperCNN, self).__init__()
    #         # Define layers here
    #     
    #     def forward(self, x):
    #         # Define forward pass
    #         return x
    
    # TODO: Define data augmentation function
    # def augment_batch(images_tensor):
    #     # Apply random horizontal flip and random crop
    #     return augmented_images
    
    # TODO: Convert numpy arrays to PyTorch tensors and create DataLoader
    # X_train_tensor = torch.from_numpy(train_images).float().permute(0, 3, 1, 2).to(device)
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # TODO: Initialize model, loss, optimizer, and scheduler
    # model = ProperCNN().to(device)
    # criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    # optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # TODO: Training loop with augmentation and gradient clipping
    # for epoch in range(num_epochs):
    #     for X_batch, y_batch in train_loader:
    #         X_batch_aug = augment_batch(X_batch)
    #         # ... forward, backward, clip gradients, optimizer step
    #     scheduler.step()
    
    # TODO: Evaluate on test set and return accuracy
    
    raise NotImplementedError


In [ ]:
# Evaluate the deeper CNN implementation
res = test_exercise_7_proper_cnn(student_proper_cnn)
show_result("Exercise 3 – Proper CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_proper_cnn(X_train, y_train, X_test, y_test)
    print(f"Proper CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_proper_cnn above.")


## 5. Discussion

Briefly reflect on your results:

- Did the deeper CNN outperform the baseline models? By how much?
- What were the key techniques that helped achieve >50% accuracy?
- Why is it important to compare against simple baselines?
- How does PyTorch compare to implementing CNNs from scratch with NumPy?

_Provide your answers here._
